# Trace Cross Encoder

In [1]:
model_id = "cross-encoder/stsb-roberta-base"
test_sentences = [('How many people live in Berlin?', 'How many people live in Berlin?'), ('Berlin has a population of 3,520,031 registered inhabitants in an area of 891.82 square kilometers.', 'New York City is famous for the Metropolitan Museum of Art.')]

In [2]:
import torch
from sentence_transformers import CrossEncoder
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch

model = AutoModelForSequenceClassification.from_pretrained(model_id)
model.eval()
tokenizer = AutoTokenizer.from_pretrained(model_id)
features = tokenizer([('Query', 'Paragraph1'), ('Query', 'Paragraph2') , ('Query', 'Paragraph3')],  padding=True, truncation=True, return_tensors="pt")

In [3]:
features

{'input_ids': tensor([[    0, 48382,     2,     2, 22011, 44947,   134,     2],
        [    0, 48382,     2,     2, 22011, 44947,   176,     2],
        [    0, 48382,     2,     2, 22011, 44947,   246,     2]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1, 1, 1, 1]])}

In [5]:
ex_input = (features["input_ids"], features["attention_mask"])
traced_model = torch.jit.trace(model, ex_input, strict=False)
torch.jit.save(traced_model, "traced_cross_encoder.pt")

In [6]:
loaded_model = torch.jit.load("traced_cross_encoder.pt")
loaded_model.eval()

RecursiveScriptModule(
  original_name=RobertaForSequenceClassification
  (roberta): RecursiveScriptModule(
    original_name=RobertaModel
    (embeddings): RecursiveScriptModule(
      original_name=RobertaEmbeddings
      (word_embeddings): RecursiveScriptModule(original_name=Embedding)
      (position_embeddings): RecursiveScriptModule(original_name=Embedding)
      (token_type_embeddings): RecursiveScriptModule(original_name=Embedding)
      (LayerNorm): RecursiveScriptModule(original_name=LayerNorm)
      (dropout): RecursiveScriptModule(original_name=Dropout)
    )
    (encoder): RecursiveScriptModule(
      original_name=RobertaEncoder
      (layer): RecursiveScriptModule(
        original_name=ModuleList
        (0): RecursiveScriptModule(
          original_name=RobertaLayer
          (attention): RecursiveScriptModule(
            original_name=RobertaAttention
            (self): RecursiveScriptModule(
              original_name=RobertaSelfAttention
              (query): R

In [7]:
test_features = tokenizer(test_sentences,  padding=True, truncation=True, return_tensors="pt")
pt_prediction = loaded_model(**test_features)
pt_prediction

{'logits': tensor([[ 5.6635],
         [-3.7582]], grad_fn=<AddmmBackward0>)}

In [8]:
from torch import nn
default_activation_function = nn.Sigmoid() #if config.num_labels == 1 else nn.Identity()
activation_fct = default_activation_function
logits = activation_fct(pt_prediction['logits'])

In [9]:
logits

tensor([[0.9965],
        [0.0228]], grad_fn=<SigmoidBackward0>)

# Compare Output

In [10]:
from sentence_transformers import CrossEncoder
model = CrossEncoder(model_id)
original_embedding = model.predict(test_sentences)
original_embedding

array([0.99654156, 0.02279292], dtype=float32)

In [11]:
from transformers import AutoConfig, AutoTokenizer, AutoModelForSequenceClassification
import torch

config = AutoConfig.from_pretrained(model_id)
model = AutoModelForSequenceClassification.from_pretrained(model_id, config=config)
tokenizer = AutoTokenizer.from_pretrained(model_id)
max_length = 512

In [12]:
from torch.utils.data import DataLoader

def smart_batching_collate_text_only(batch):
    texts = [[] for _ in range(len(batch[0]))]

    for example in batch:
        for idx, text in enumerate(example):
            texts[idx].append(text.strip())

    tokenized = tokenizer(*texts, padding=True, truncation='longest_first', return_tensors="pt", max_length=max_length)

    for name in tokenized:
        tokenized[name] = tokenized[name].to('cpu')

    return tokenized

inp_dataloader = DataLoader(test_sentences, batch_size=32, collate_fn=smart_batching_collate_text_only, num_workers=0, shuffle=False)

In [13]:
inp_dataloader

In [14]:
from torch import nn
default_activation_function = nn.Sigmoid() #if config.num_labels == 1 else nn.Identity()

In [15]:
iterator = inp_dataloader
activation_fct = None

if activation_fct is None:
    activation_fct = default_activation_function

pred_scores = []
model.eval()
model.to('cpu')
with torch.no_grad():
    for features in iterator:
        model_predictions = model(**features, return_dict=True)
        logits = activation_fct(model_predictions.logits)

#         if apply_softmax and len(logits[0]) > 1:
#             logits = torch.nn.functional.softmax(logits, dim=1)
        pred_scores.extend(logits)

In [16]:
pred_scores

[tensor([0.9965]), tensor([0.0228])]